In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('chinook.db')

In [ ]:
# Homework 1
# Task 1

customers_df = pd.read_sql_query("SELECT CustomerId, FirstName, LastName FROM Customer", conn)
invoices_df = pd.read_sql_query("SELECT CustomerId, Total FROM Invoice", conn)

customer_spending = invoices_df.groupby('CustomerId')['Total'].sum().reset_index()

result_df = customer_spending.merge(customers_df, on='CustomerId')

result_df = result_df[['CustomerId', 'FirstName', 'LastName', 'Total']]
result_df = result_df.sort_values(by='Total', ascending=False).reset_index(drop=True)

print(result_df)

conn.close()

    CustomerId  FirstName      LastName  Total
0            6     Helena          Holý  49.62
1           26    Richard    Cunningham  47.62
2           57       Luis         Rojas  46.62
3           45   Ladislav        Kovács  45.62
4           46       Hugh      O'Reilly  45.62
5           28      Julia       Barnett  43.62
6           37       Fynn    Zimmermann  43.62
7           24      Frank       Ralston  43.62
8            7     Astrid        Gruber  42.62
9           25     Victor       Stevens  42.62
10          44      Terhi    Hämäläinen  41.62
11           5  František   Wichterlová  40.62
12          43   Isabelle       Mercier  40.62
13          48   Johannes  Van der Berg  40.62
14           1       Luís     Gonçalves  39.62
15          17       Jack         Smith  39.62
16          22    Heather       Leacock  39.62
17          34       João     Fernandes  39.62
18          42      Wyatt        Girard  39.62
19          20        Dan        Miller  39.62
20           

In [ ]:
# Task 2

conn = sqlite3.connect('chinook.db')

customers_df = pd.read_sql_query("SELECT CustomerId, FirstName, LastName FROM Customer", conn)
invoices_df = pd.read_sql_query("SELECT CustomerId, Total FROM Invoice", conn)

customer_spending = invoices_df.groupby('CustomerId')['Total'].sum().reset_index()

result_df = customer_spending.merge(customers_df, on='CustomerId')

top_5_customers = result_df.sort_values(by='Total', ascending=False).head(5)

top_5_customers = top_5_customers[['CustomerId', 'FirstName', 'LastName', 'Total']].reset_index(drop=True)

print(top_5_customers)

conn.close()

   CustomerId FirstName    LastName  Total
0           6    Helena        Holý  49.62
1          26   Richard  Cunningham  47.62
2          57      Luis       Rojas  46.62
3          45  Ladislav      Kovács  45.62
4          46      Hugh    O'Reilly  45.62


In [ ]:
# Task 3

conn = sqlite3.connect('chinook.db')

invoices_df = pd.read_sql_query("SELECT CustomerId, Total FROM Invoice", conn)
customers_df = pd.read_sql_query("SELECT CustomerId, FirstName || ' ' || LastName AS FullName FROM Customer", conn)

total_spent = invoices_df.groupby('CustomerId')['Total'].sum().reset_index()

top_customers = total_spent.merge(customers_df, on='CustomerId')

top_5 = top_customers.sort_values(by='Total', ascending=False).head(5).reset_index(drop=True)

print(top_5[['CustomerId', 'FullName', 'Total']])

conn.close()

   CustomerId            FullName  Total
0           6         Helena Holý  49.62
1          26  Richard Cunningham  47.62
2          57          Luis Rojas  46.62
3          45     Ladislav Kovács  45.62
4          46       Hugh O'Reilly  45.62


In [ ]:
# Homework 2
# Task 1

conn = sqlite3.connect('chinook.db')

invoice_lines = pd.read_sql_query("SELECT InvoiceId, TrackId FROM InvoiceLine", conn)
tracks = pd.read_sql_query("SELECT TrackId, AlbumId FROM Track", conn)
invoices = pd.read_sql_query("SELECT InvoiceId, CustomerId FROM Invoice", conn)

invoice_album = invoice_lines.merge(tracks, on='TrackId')
invoice_album = invoice_album.merge(invoices, on='InvoiceId')

album_track_counts = tracks.groupby('AlbumId')['TrackId'].nunique().reset_index()
album_track_counts.rename(columns={'TrackId': 'TotalTracks'}, inplace=True)

invoice_album_counts = invoice_album.groupby(['InvoiceId', 'AlbumId'])['TrackId'].count().reset_index()
invoice_album_counts = invoice_album_counts.merge(album_track_counts, on='AlbumId')

invoice_album_counts['FullAlbum'] = invoice_album_counts['TrackId'] == invoice_album_counts['TotalTracks']

invoice_album = invoice_album.merge(invoice_album_counts[['InvoiceId', 'AlbumId', 'FullAlbum']], on=['InvoiceId', 'AlbumId'])
album_buyers = invoice_album[invoice_album['FullAlbum'] == True]['CustomerId'].unique()

all_customers = invoices['CustomerId'].unique()

track_buyers = set(all_customers) - set(album_buyers)

track_buyer_percentage = len(track_buyers) / len(all_customers) * 100

print(f"Percentage of customers who prefer individual track purchases: {track_buyer_percentage:.2f}%")

# Close connection
conn.close()

Percentage of customers who prefer individual track purchases: 64.41%


In [8]:
# Task 2

conn = sqlite3.connect('chinook.db')

tracks = pd.read_sql_query("SELECT TrackId, AlbumId FROM Track", conn)
invoice_lines = pd.read_sql_query("SELECT InvoiceId, TrackId FROM InvoiceLine", conn)
invoices = pd.read_sql_query("SELECT InvoiceId, CustomerId FROM Invoice", conn)

invoice_album = invoice_lines.merge(tracks, on='TrackId')
invoice_album = invoice_album.merge(invoices, on='InvoiceId')

album_track_counts = tracks.groupby('AlbumId')['TrackId'].nunique().reset_index()
album_track_counts.rename(columns={'TrackId': 'AlbumTotalTracks'}, inplace=True)

invoice_album_counts = invoice_album.groupby(['InvoiceId', 'AlbumId']).agg({'TrackId': 'count'}).reset_index()
invoice_album_counts.rename(columns={'TrackId': 'TracksPurchased'}, inplace=True)

invoice_album_counts = invoice_album_counts.merge(album_track_counts, on='AlbumId')
invoice_album_counts['FullAlbum'] = invoice_album_counts['TracksPurchased'] == invoice_album_counts['AlbumTotalTracks']

invoice_customer_map = invoices[['InvoiceId', 'CustomerId']]
invoice_album_counts = invoice_album_counts.merge(invoice_customer_map, on='InvoiceId')

album_buyers = invoice_album_counts[invoice_album_counts['FullAlbum'] == True]['CustomerId'].unique()

all_customers = pd.read_sql_query("SELECT CustomerId FROM Customer", conn)['CustomerId'].unique()

track_preferring_customers = set(all_customers) - set(album_buyers)

track_preferring_pct = (len(track_preferring_customers) / len(all_customers)) * 100

print(f"Percentage of customers who prefer individual tracks (never bought a full album): {track_preferring_pct:.2f}%")

conn.close()

Percentage of customers who prefer individual tracks (never bought a full album): 64.41%


In [ ]:
# task 3

conn = sqlite3.connect('chinook.db')

tracks = pd.read_sql_query("SELECT TrackId, AlbumId FROM Track", conn)
invoice_lines = pd.read_sql_query("SELECT InvoiceId, TrackId FROM InvoiceLine", conn)
invoices = pd.read_sql_query("SELECT InvoiceId, CustomerId FROM Invoice", conn)
customers = pd.read_sql_query("SELECT CustomerId FROM Customer", conn)

invoice_album = invoice_lines.merge(tracks, on='TrackId')
invoice_album = invoice_album.merge(invoices, on='InvoiceId')

album_track_counts = tracks.groupby('AlbumId')['TrackId'].nunique().reset_index()
album_track_counts.rename(columns={'TrackId': 'AlbumTotalTracks'}, inplace=True)

invoice_album_counts = invoice_album.groupby(['InvoiceId', 'AlbumId']).agg({'TrackId': 'count'}).reset_index()
invoice_album_counts.rename(columns={'TrackId': 'TracksPurchased'}, inplace=True)
invoice_album_counts = invoice_album_counts.merge(album_track_counts, on='AlbumId')

invoice_album_counts['FullAlbum'] = invoice_album_counts['TracksPurchased'] == invoice_album_counts['AlbumTotalTracks']

invoice_album_counts = invoice_album_counts.merge(invoices[['InvoiceId', 'CustomerId']], on='InvoiceId')

full_album_buyers = set(invoice_album_counts[invoice_album_counts['FullAlbum']]['CustomerId'])

all_customers = set(customers['CustomerId'])

track_only_buyers = all_customers - full_album_buyers

total_customers = len(all_customers)
num_album_buyers = len(full_album_buyers)
num_track_buyers = len(track_only_buyers)

album_buyer_pct = (num_album_buyers / total_customers) * 100
track_buyer_pct = (num_track_buyers / total_customers) * 100

summary_df = pd.DataFrame({
    'CustomerType': ['Full Album Buyers', 'Track-Only Buyers'],
    'CustomerCount': [num_album_buyers, num_track_buyers],
    'Percentage': [album_buyer_pct, track_buyer_pct]
})

print(summary_df)

conn.close()

        CustomerType  CustomerCount  Percentage
0  Full Album Buyers             21    35.59322
1  Track-Only Buyers             38    64.40678
